# Debug LLMs with W&B Trace and LangChain

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/trace_debugging/trace_quickstart_langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Use W&B Traces with LangChain to
* trace and visualize all the intermediate processing steps of your LLM calls
* debug and diagnose errors in specific prompts or chain configurations
* quickly find more effective prompts and LLM chains for your specific use case 

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/traces_debug_board.png">

# Step 0: Setup

Install dependencies, authenticate with OpenAI, and login to W&B so you can save and share your work.

In [ ]:
!pip install -qqq wandb langchain==0.0.295
# if not already installed
#!pip install -qqq weave

In [ ]:
import os
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

In [ ]:
import wandb
wandb.login()

In [ ]:
import langchain
import weave
from weave.monitoring.langchain import WeaveTracer

# Step 1: Configure data streaming and storage in W&B

In [ ]:
WB_ENTITY = # replace with your W&B username or team name
WB_PROJECT = "lc_trace_quickstart" # top-level directory for this work
WB_STREAM = "prompts" # record table which stores the logs of LangChain calls

In [ ]:
# feel free to edit or add your own!
sample_math_questions = [
   'What is the sum of 5 and 7?',
   'What is the product of 4 and 6?',
   'What is the square root of 64?',
   'What is the area of a circle with a radius of 5?',
   'What is the value of x in the equation 3x + 4 = 10?',
   'What is the value of y in the equation y = 2x + 1 when x = 3?',
   'What is the equation of the line that passes through the points (2, 4) and (3, 6)?',
   'What is the volume of a cube with a side length of 5?',
   'What is the value of x in the equation x^2 + 3x - 4 = 0?'
]

# Step 2: Add a WeaveTracer as a callback

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# store all traces in the StreamTable we configured above
tracer = WeaveTracer(f"{WB_ENTITY}/{WB_PROJECT}/{WB_STREAM}")

for question in sample_math_questions:
    try:
        # add WeaveTracer as a callback to monitor & log all calls
        print(agent.run(question, callbacks=[tracer]))
    except Exception as e:
        print("Caught Exception:", e)

# Step 3: Create and customize a Trace Debugging Board

Click on the "View data at:" link above to view your data stream in Weave.

You can click on "+ New board from template" on the right to create a Trace Debug Board, which enables:
* key LLM tuning metrics at a glance**: latency and success vs failure, for each call and as a distribution
* complete view of call details: inputs, outputs, status, timestamp, etc&mdash;also available for downtream queries & analaysis
* interactive W&B Trace view: a color-coded flow chart of every step of an LLM chain, with full metadata, model details, and similar span views updating in sync with every selected span
* monitor & analyze from the automatic Board; customize & publish the Board for streamlined collaboration with your team